<a href="https://colab.research.google.com/github/sarang330/_PROJECTS_AND_LABS_SARANG/blob/main/Codebase_for_Comprehensive_Reportipynb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Codebase and Report on aspects of Deep Learning**

For introduction **Deep Learning** is a subset of machine learning based on artificial neural networks with representation learning. It involves training models to recognize patterns and make decisions with minimal human intervention. Deep learning has shown remarkable performance in tasks such as **image recognition**, **speech processing**, and **natural language understanding** due to its ability to learn from vast amounts of data.

In [ ]:
pip install torch torchvision transformers

  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl (176.2 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (99 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
import torchvision.datasets as datasets
from torch.utils.data import DataLoader

In [ ]:
# Define the device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
# Hyperparameters
batch_size = 64
learning_rate = 0.001
num_epochs = 10


In [ ]:
# Transformations
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),])

In [ ]:
# CIFAR-10 Dataset
train_dataset = datasets.CIFAR10(root='./data', train=True, transform=transform, download=True)
test_dataset = datasets.CIFAR10(root='./data', train=False, transform=transform, download=True)


Files already downloaded and verified
Files already downloaded and verified


In [ ]:
train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=False)

In [ ]:
# Helper function to calculate accuracy
def calculate_accuracy(loader, model):
    model.eval()
    num_correct = 0
    num_samples = 0
    with torch.no_grad():
        for x, y in loader:
            x = x.to(device)
            y = y.to(device)
            scores = model(x)
            _, predictions = scores.max(1)
            num_correct += (predictions == y).sum()
            num_samples += predictions.size(0)
    model.train()
    return float(num_correct) / float(num_samples) * 100


### **Convolutional Neural Networks {CNNs}**


CNNs are specialized neural networks designed to process and analyze visual data, such as images and videos. They leverage spatial hierarchies in data through convolutional layers, pooling layers, and fully connected layers.

1.	Convolutional Layers

2.	Pooling Layers

3.	Fully Connected Layers

In [ ]:
# CNN Model
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, stride=1, padding=1)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2, padding=0)
        self.fc1 = nn.Linear(64 * 8 * 8, 128)
        self.fc2 = nn.Linear(128, 10)
        self.relu = nn.ReLU()

    def forward(self, x):
        x = self.pool(self.relu(self.conv1(x)))
        x = self.pool(self.relu(self.conv2(x)))
        x = x.view(-1, 64 * 8 * 8)
        x = self.relu(self.fc1(x))
        x = self.fc2(x)
        return x


### **Recurrent Neural Networks {RNNs}**


RNNs are designed to handle sequential data by maintaining a memory of previous inputs, making them suitable for tasks such as time series prediction and natural language processing.

1.	Basic RNN

2.	Long Short-Term Memory (LSTM)

3.	Gated Recurrent Unit (GRU)


In [ ]:

# RNN Model
class RNN(nn.Module):
    def __init__(self, input_size=32*32*3, hidden_size=128, num_layers=2, num_classes=10):
        super(RNN, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.rnn = nn.RNN(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, num_classes)

    def forward(self, x):
        x = x.view(x.size(0), -1, 32*32*3)
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(device)
        out, _ = self.rnn(x, h0)
        out = self.fc(out[:, -1, :])
        return out


### **Transformers**

GRUs are computationally simpler and often perform similarly to LSTMs, but they may not capture dependencies as effectively in some cases.

1.	Attention Mechanism

2.	Encoder-Decoder Architecture

In [ ]:
# Transformer Model
class TransformerModel(nn.Module):
    def __init__(self, input_size=32*32*3, num_classes=10):
        super(TransformerModel, self).__init__()
        self.encoder_layer = nn.TransformerEncoderLayer(d_model=input_size, nhead=8)
        self.transformer_encoder = nn.TransformerEncoder(self.encoder_layer, num_layers=2)
        self.fc = nn.Linear(input_size, num_classes)

    def forward(self, x):
        x = x.view(x.size(0), 32*32*3)
        x = self.transformer_encoder(x.unsqueeze(0)).squeeze(0)
        x = self.fc(x)
        return x


In [ ]:
# Training function
def train_model(model, train_loader, optimizer, criterion, num_epochs):
    model = model.to(device)
    for epoch in range(num_epochs):
        for batch_idx, (data, targets) in enumerate(train_loader):
            data = data.to(device)
            targets = targets.to(device)
            scores = model(data)
            loss = criterion(scores, targets)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
        print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}")



In [ ]:
# CNN Training and Evaluation
cnn = CNN()
optimizer = optim.Adam(cnn.parameters(), lr=learning_rate)
criterion = nn.CrossEntropyLoss()
train_model(cnn, train_loader, optimizer, criterion, num_epochs)
cnn_acc = calculate_accuracy(test_loader, cnn)
print(f'CNN Test Accuracy: {cnn_acc:.2f}%')



Epoch [1/10], Loss: 0.6174
Epoch [2/10], Loss: 0.9911
Epoch [3/10], Loss: 0.8834
Epoch [4/10], Loss: 0.6959
Epoch [5/10], Loss: 0.7029
Epoch [6/10], Loss: 0.3154
Epoch [7/10], Loss: 0.2437
Epoch [8/10], Loss: 0.6621
Epoch [9/10], Loss: 0.2165
Epoch [10/10], Loss: 0.0452
CNN Test Accuracy: 71.38%


In [ ]:
# RNN Training and Evaluation
rnn = RNN()
optimizer = optim.Adam(rnn.parameters(), lr=learning_rate)
criterion = nn.CrossEntropyLoss()
train_model(rnn, train_loader, optimizer, criterion, num_epochs)
rnn_acc = calculate_accuracy(test_loader, rnn)
print(f'RNN Test Accuracy: {rnn_acc:.2f}%')


Epoch [1/10], Loss: 2.0190
Epoch [2/10], Loss: 1.1782
Epoch [3/10], Loss: 1.4935
Epoch [4/10], Loss: 1.8005
Epoch [5/10], Loss: 1.4827
Epoch [6/10], Loss: 1.4851
Epoch [7/10], Loss: 1.7581
Epoch [8/10], Loss: 1.1708
Epoch [9/10], Loss: 0.8146
Epoch [10/10], Loss: 1.4851
RNN Test Accuracy: 48.39%


In [ ]:
# Transformer Training and Evaluation
transformer = TransformerModel()
optimizer = optim.Adam(transformer.parameters(), lr=learning_rate)
criterion = nn.CrossEntropyLoss()
train_model(transformer, train_loader, optimizer, criterion, num_epochs)
transformer_acc = calculate_accuracy(test_loader, transformer)
print(f'Transformer Test Accuracy: {transformer_acc:.2f}%')


/usr/local/lib/python3.10/dist-packages/torch/nn/modules/transformer.py:306: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(f"enable_nested_tensor is True, but self.use_nested_tensor is False because {why_not_sparsity_fast_path}")


Epoch [1/10], Loss: 2.0060
Epoch [2/10], Loss: 2.3065
Epoch [3/10], Loss: 1.6423
Epoch [4/10], Loss: 1.7482
Epoch [5/10], Loss: 1.7153
Epoch [6/10], Loss: 1.7266
Epoch [7/10], Loss: 1.7702
Epoch [8/10], Loss: 1.6115
Epoch [9/10], Loss: 1.7640
Epoch [10/10], Loss: 1.7084
Transformer Test Accuracy: 40.11%


### **Summary**

The provided codebase demonstrates how to:

Load and preprocess the CIFAR-10 dataset.

Define CNN, RNN, and Transformer models.

Train each model using a specified training function.

Evaluate the accuracy of each model on the test dataset.
